In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
x = tf.placeholder("float", [None, 28, 28])
y = tf.placeholder("float", [None, 10])

In [3]:
def RNN(x):
    x = tf.unstack(x, 28, 1)

    # LSTMセルを定義する
    lstm_cell = rnn.BasicLSTMCell(128, forget_bias=1.0)

    # モデルの定義。各タイムステップの出力値と状態が返される
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # 変数定義
    weight = tf.Variable(tf.random_normal([128, 10]))
    bias = tf.Variable(tf.random_normal([10]))

    return tf.matmul(outputs[-1], weight) + bias

preds = RNN(x)

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').


In [4]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=preds, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

# 評価用
correct_pred = tf.equal(tf.argmax(preds, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [5]:
batch_size = 128
n_training_iters = 100000
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < n_training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # next_batchで返されるbatch_xは[batch_size, 784]のテンソルなので、batch_size×28×28に変換します。
        batch_x = batch_x.reshape((batch_size, 28, 28))
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % 10 == 0:
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            print('step: {} / loss: {:.6f} / acc: {:.5f}'.format(step, loss, acc))
        step += 1

    # テスト
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, 28, 28))
    test_label = mnist.test.labels[:test_len]
    test_acc = sess.run(accuracy, feed_dict={x: test_data, y: test_label})
    print("Test Accuracy: {}".format(test_acc))

step: 10 / loss: 1.975351 / acc: 0.28906
step: 20 / loss: 1.772740 / acc: 0.39844
step: 30 / loss: 1.557635 / acc: 0.50000
step: 40 / loss: 1.126119 / acc: 0.60938
step: 50 / loss: 1.171542 / acc: 0.60156
step: 60 / loss: 0.806178 / acc: 0.71094
step: 70 / loss: 0.777341 / acc: 0.74219
step: 80 / loss: 0.483344 / acc: 0.82812
step: 90 / loss: 0.492552 / acc: 0.83594
step: 100 / loss: 0.776954 / acc: 0.74219
step: 110 / loss: 0.429149 / acc: 0.84375
step: 120 / loss: 0.451305 / acc: 0.85938
step: 130 / loss: 0.387711 / acc: 0.87500
step: 140 / loss: 0.429635 / acc: 0.87500
step: 150 / loss: 0.288445 / acc: 0.87500
step: 160 / loss: 0.319236 / acc: 0.89844
step: 170 / loss: 0.355023 / acc: 0.92188
step: 180 / loss: 0.345884 / acc: 0.88281
step: 190 / loss: 0.249111 / acc: 0.92969
step: 200 / loss: 0.273456 / acc: 0.90625
step: 210 / loss: 0.349582 / acc: 0.89844
step: 220 / loss: 0.235096 / acc: 0.91406
step: 230 / loss: 0.324970 / acc: 0.89062
step: 240 / loss: 0.410949 / acc: 0.87500
s